In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!unzip '/content/drive/MyDrive/yolo_plate_ocr_dataset' -d '/content/yolo_plate_ocr_dataset'

  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay596.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay598.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay60.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay600.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay604.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay61.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay612.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay633.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay634.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay636.jpg  
  inflating: /content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/3xemay637.jpg  
 

In [3]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [4]:
import pandas as pd
import numpy as np
import cv2
import albumentations as A
from decimal import Decimal, ROUND_HALF_UP
import glob

In [5]:
def get_boxes(path):
  txt = pd.read_csv(path, delimiter=' ', header=None)
  arr_txt = np.asarray(txt)
  new_arr = np.zeros_like(arr_txt)
  new_arr[:, :4] = arr_txt[:, 1:]
  new_arr[:, 4] = arr_txt[:, 0]
  return new_arr.tolist()

def get_images(path):
  return cv2.imread(path)

transform = A.Compose([
    A.Rotate(limit=40, p=1),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.7))


def write_array_to_text_file(list_array, save_path):
    array = np.array(list_array)
    # Làm tròn các giá trị số thập phân
    rounded_array = np.zeros_like(array, dtype=object)
    for i in range(array.shape[0]):
      for j in range(array.shape[1]-1):
        rounded_array[i, j+1] = Decimal(array[i, j]).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP)

    for i in range(array.shape[0]):
      rounded_array[i, 0] = Decimal(array[i, 4]).quantize(Decimal('0.'), rounding=ROUND_HALF_UP)

    # Ghi mảng vào file văn bản
    with open(save_path, 'w') as file:
      for row in rounded_array:
        line = ' '.join(map(str, row)) + '\n'
        file.write(line)


In [6]:
!mkdir /content/augment
!mkdir /content/augment/labels
!mkdir /content/augment/images

In [7]:
a = glob.glob('/content/yolo_plate_ocr_dataset/yolo_plate_ocr_dataset/images/*')
for image_path in a:
  label_path = image_path[:55] + 'labels/' + image_path[62:len(image_path)-3] + 'txt'
  augment_image = transform(image=get_images(image_path), bboxes=get_boxes(label_path))
  new_image_path = '/content/augment/images/' + 'rotate' + image_path[62:len(image_path)-3] + 'jpg'
  new_label_path = '/content/augment/labels/' + 'rotate' + image_path[62:len(image_path)-3] + 'txt'
  cv2.imwrite(new_image_path, augment_image['image'])
  write_array_to_text_file(augment_image['bboxes'],new_label_path)

In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!zip -r /content/augment1 /content/augment

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  adding: content/augment/labels/rotate5xemay767.txt (deflated 53%)
  adding: content/augment/labels/rotate4xemay1955.txt (deflated 52%)
  adding: content/augment/labels/rotate13xemay1343.txt (deflated 51%)
  adding: content/augment/labels/rotateiwt350.txt (deflated 52%)
  adding: content/augment/labels/rotateiwt83.txt (deflated 47%)
  adding: content/augment/labels/rotateiwt410.txt (deflated 38%)
  adding: content/augment/labels/rotate3xemay1305.txt (deflated 52%)
  adding: content/augment/labels/rotate6xemay97.txt (deflated 53%)
  adding: content/augment/labels/rotateiwt344.txt (deflated 51%)
  adding: content/augment/labels/rotate8xemay1182.txt (deflated 52%)
  adding: content/augment/labels/rotate2xemay113.txt (deflated 52%)
  adding: content/augment/labels/rotate4xemay2037.txt (deflated 52%)
  adding: content/augment/images/rotate3xemay744.jpg (deflated 0%)
  adding: content/augment/images/rotate3xemay1327.jpg (deflated 0%)
